In [1]:
from rosemary import jpt_parse_args, jpt_setup, jpt_in_notebook; jpt_setup()

if jpt_in_notebook():
    import os
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    os.environ['CUDA_VISIBLE_DEVICES'] = \
        os.environ['CUDA_VISIBLE_DEVICES'].split(',')[1]
    # '0,1,2,3,4,5'
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    
!nvidia-smi

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


4,5
5
Wed Nov 22 23:01:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   33C    P0    64W / 300W |  31643MiB / 32510MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| 

In [2]:
import os
import itertools
from rosemary import jpt_in_notebook
from llm.submit import submit_job, multiline_to_singleline, shell_scripts_template_slurm

log_dir = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/'

test_run = 1
test_run = bool(test_run)

# model_name = 'llama-7b'; encode_fn_type = 'sft'; md = 'llama7b'
# model_name = 'llama-7b+lora:r=256:a=256'; encode_fn_type = 'sft'; md = 'llama7b'
# model_name = 'mistral-7b+lora:r=256:a=256'; encode_fn_type = 'sft'; md = 'mistral7b'
model_name = 'all-mpnet-base-v2'; encode_fn_type = 'input'; md = 'mpnet'
# model_name = 'bge-large-en-v1.5'; encode_fn_type = 'input'; md = 'bge'


sort_by_list = [
    f'dppmapbd_nc=200_k=vmf_gamma={gamma}_kmd=mpnet' 
    for gamma in [.3, 1., 3., 10., 15.]] # [.3, 1., 3., 10., 15.]
# sort_by_list += [f'dppmapbd_nc=200_k=lin_kmd=mpnet']
# sort_by_list = [
#     f'dppmap_theta={theta}_k=vmf_gamma=3.0_kmd=mpnet_q=log+prob_qmd=mistral7b' for theta in [0, .3, .6, .9, .95]
# ]


# nc_list = [100, 500, 1000]
# nc_list = [100, 200, 300, 400, 500, 600]
# sort_by_list = [
#     f'semdedup_cl=kmeansfaisscd_md={md}_dist=cd_emb={emb}_nc={nc}'
#     for nc in nc_list
#     for emb in ['text+embedding', 'grad+rp+loraB'] # 'grad+rp+loraB' 
# ]

# sort_by_list = [
#     'random_s=0', 'random_s=1', 'random_s=2',
#     'log_prob', 'logit_margin', 'el2n_agg=mean',
#     'grad_loraB_l2n', 'numtoks'
# ]
# sort_by_list += [
# #     'kmeansl2_emb=grad+rp+loraB_nc=300',
# #     'kmeansl2_emb=grad+rp+loraB_nc=1000',
#     'kmeansl2_emb=grad+rp+loraB_nc=3000',
# #     'kmeansl2_emb=text+embedding_nc=300',
#     'kmeansl2_emb=text+embedding_nc=1000',
# #     'kmeansl2_emb=text+embedding_nc=3000',
# ]
# sort_by_list = ['numtoks']

# sort_by_list += [
#     'dppmap_emb=grad+rp+loraB_k=Kcos', 
#     'dppmap_emb=text+embedding_k=Kcos', 
#     'dppmap_emb=grad+rp+loraB_k=Kcosp', 
#     'dppmap_emb=text+embedding_k=Kcosp',
#     'dppmap_emb=grad+rp+loraB_k=Kcos1np', 
#     'dppmap_emb=text+embedding_k=Kcos1np',
# ]
# dataset_list = ['lima']
# dataset_list = ['flan2022_1m']
# dataset_list = ['tulu_v1_mix']
# dataset_list = ['ultrachat']
dataset_list = ['ultrachat15']
# dataset_list = ['wizardlm']
# dataset_list = ['sharegpt']


# sort_by_list = [
#  'log_prob',
#  'el2n_agg=mean',
#  'el2n_agg=l2n',
#  'logit_margin',
# ]
# if 'lora' in model_name:
#     sort_by_list += ['grad_loraB_l2n']
# else:
#     sort_by_list += ['grad_all_l2n', 'grad_qkv_l2n', 'grad_mlp_l2n', 'grad_last_l2n',]
# sort_by_list = ['kmeansl2_emb=grad+rp+loraB_nc=30',
#                 'kmeansl2_emb=text+embedding_nc=30']
# dataset_list = ['lima']


# model_name = 'pythia-1b-deduped'
# model_name = 'pythia-1b-deduped+lora:r=256:a=256'
# dataset_list = ['cot', 'dolly', 'flan_v2', 'lima', 'oasst1']
# # sort_by_list = ['random_s=0', 
# #                 'log_prob', 'logit_margin', 'el2n_agg=mean', 'el2n_agg=l2n', 
# #                 'kmeansl2_nc=3000', 'kmeanscd_nc=3000',
# #                 'grad_loraB_l2n',
# #                 'grad_all_l2n', 'grad_qkv_l2n', 'grad_mlp_l2n', 'grad_last_l2n',
# #                ]
# sort_by_list = ['grad_loraB_l2n']

from note_pruning_analysis import data_inds_dir

options_list = itertools.product(dataset_list, sort_by_list)

print('test_run =',test_run)
cmds = []
for dataset, sort_by in options_list:
    save_dir = os.path.join(data_inds_dir, model_name, dataset)
    cmd = f"""
     python note_pruning.py \
        --dataset {dataset} \
        --sort_by {sort_by} \
        --model_name {model_name} \
        --encode_fn_type {encode_fn_type} \
        --save_dir {save_dir} \
    """.strip()
    cmd = multiline_to_singleline(cmd)
    shell_scripts = shell_scripts_template_slurm.format(
        conda_env='open-instruct',
        cwd=os.getcwd(),
        cmd=cmd,
        log_dir=log_dir,
        save_dir=save_dir)
    out = submit_job(
        shell_scripts, 
        job_name=f'prune.{dataset}.{sort_by}', 
        nodes=1,
        num_cpus=64, # 32
        cpu_mem=256, # 128
        num_gpus=1,
        gpu_type='v100',
        test_run=test_run,
        job_duration=6,
    )
    cmds.append(cmd)
    print(cmd)
        
print('#cmds: ', len(cmds))


test_run = False

Submiting job with:
{
    "job_name": "prune.ultrachat15.dppmapbd_nc=200_k=vmf_gamma=0.3_kmd=mpnet",
    "nodes": 1,
    "num_cpus": 64,
    "cpu_mem": 256,
    "num_gpus": 1,
    "gpu_type": "v100",
    "test_run": false,
    "queue": "el8",
    "num_jobs": 1
}
python note_pruning.py --dataset ultrachat15 --sort_by dppmapbd_nc=200_k=vmf_gamma=0.3_kmd=mpnet --model_name all-mpnet-base-v2 --encode_fn_type input --save_dir /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data_inds/all-mpnet-base-v2/ultrachat15

Submiting job with:
{
    "job_name": "prune.ultrachat15.dppmapbd_nc=200_k=vmf_gamma=1.0_kmd=mpnet",
    "nodes": 1,
    "num_cpus": 64,
    "cpu_mem": 256,
    "num_gpus": 1,
    "gpu_type": "v100",
    "test_run": false,
    "queue": "el8",
    "num_jobs": 1
}
python note_pruning.py --dataset ultrachat15 --sort_by dppmapbd_nc=200_k=vmf_gamma=1.0_kmd=mpnet --model_name all-mpnet-base-v2 --encode_fn_type input --save_dir /gpfs/u

In [2]:
!CUDA_VISIBLE_DEVICES=1 python note_pruning.py --dataset ultrachat15 --sort_by dppmapbd_nc=200_k=vmf_gamma=1.0_kmd=mpnet --model_name all-mpnet-base-v2 --encode_fn_type input --save_dir /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data_inds/all-mpnet-base-v2/ultrachat15


/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')
{
  "dataset": "ultrachat15",
  "sort_by": "dppmapbd_nc=200_k=vmf_gamma=0.3_kmd=mpnet",
  "model_name": "all-mpnet-base-v2",
  "save_dir": "/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data_inds/all-mpnet-base-v2/ultrachat15",
  "encode_fn_type": "input",
  "test_run": false
}
Calling note_pruning_dpp.compute_dppmap with kwargs={
    "dppmap_type": "dppmapbd",
    "dataset": "ultrachat15",
    "kernel_type": "vmf",
    "kernel_embed_model": "mpnet",
    "kernel_embed_type": "text_embedding",
    "kernel_kwargs": {
        "gamma": 0.3
    },
    "quality_score_type": null,
    "quality_score_embed_model": null,
    "theta": 0.0,
    "device": "cuda",
    "max_length": 2192
}
dppmapbd: cluster = 0 / 200
/gpfs/u/scratch/PTFM/PTFMqngp/github/mit


KeyboardInterrupt



In [ ]:
with open('note_pruning_run_cmds.sh', 'w') as f:
    s = 'set -e\nset -x\n'
    devices = os.environ['CUDA_VISIBLE_DEVICES']
    devices = 1
    s += '\n\n'.join([f"CUDA_VISIBLE_DEVICES={devices} "+x for x in cmds])
    f.write(s)

### Generate curriculum from pre-computed scores (via `note_pruning.py`)

In [11]:
import itertools
from note_curriculum import (
    get_curriculum_scores,
    get_curriculum,
    generate_curriculum,
    generate_curriculum_forall_scoring_fn,
    scores_path_to_attrs,
    np_random_choice_maximize_noreplacement,
    plt_curriculum,
)
from note_pruning_analysis import assets_dir

# model_name = 'llama-7b'; dataset = 'tulu_v1_mix'; M = 150_000
# model_name = 'llama-7b'; dataset = 'sharegpt'; M = 150_000

## mistral+ultrachat
# model_name = 'mistral-7b'; dataset = 'ultrachat200k'; M = 50_000
# model_name = 'mistral-7b'; dataset = 'ultrachat15'; M = 100_000

## semdedup
# model_name = 'llama-7b'; dataset = 'wizardlm'; M = 100_000
# model_name = 'all-mpnet-base-v2'; dataset = 'wizardlm'; M = 100_000
# model_name = 'bge-large-en-v1.5'; dataset = 'wizardlm'; M = 100_000
model_name = 'all-mpnet-base-v2'; dataset = 'ultrachat15'; M = 100_000
# model_name = 'mistral-7b'; dataset = 'ultrachat15'; M = 100_000


pacing_fn_list = [
#     f'prune_size={M}_ep=1',
    f'prune_size={M}_ep=2',
#     f'prune_size={M}_ep=3',
#     f'singlestep_size={M}_startingfrac=0.1',
#     f'singlestep_size={M}_startingfrac=0.05',
#     f'fep_size={M}_nsteps=5_startingfrac=0.05_inc=1.5',
]

output_list = generate_curriculum_forall_scoring_fn(
    model_name, dataset, pacing_fn_list, verbose=False)

save inds (length = 100000) to curriculum/all-mpnet-base-v2/ultrachat15/semdedup_cl=kmeansfaisscd_md=mpnet_dist=cd_emb=text+embedding_nc=200_neg/inds_prune_size=100000_ep=2.pkl
save inds (length = 100000) to curriculum/all-mpnet-base-v2/ultrachat15/semdedup_cl=kmeansfaisscd_md=mpnet_dist=cd_emb=text+embedding_nc=1000/inds_prune_size=100000_ep=2.pkl
save inds (length = 100000) to curriculum/all-mpnet-base-v2/ultrachat15/semdedup_cl=kmeansfaisscd_md=mpnet_dist=cd_emb=text+embedding_nc=1000_neg/inds_prune_size=100000_ep=2.pkl
save inds (length = 100000) to curriculum/all-mpnet-base-v2/ultrachat15/semdedup_cl=kmeansfaisscd_md=mpnet_dist=cd_emb=text+embedding_nc=600/inds_prune_size=100000_ep=2.pkl
save inds (length = 100000) to curriculum/all-mpnet-base-v2/ultrachat15/semdedup_cl=kmeansfaisscd_md=mpnet_dist=cd_emb=text+embedding_nc=600_neg/inds_prune_size=100000_ep=2.pkl
save inds (length = 100000) to curriculum/all-mpnet-base-v2/ultrachat15/dppmapbd_nc=200_k=lin_kmd=mpnet/inds_prune_size=1

In [ ]:
1

In [ ]:
import re
import numpy as np
from note_curriculum import get_curriculum_scores, generate_curriculum, plt_curriculum
import glob
import matplotlib.pyplot as plt

model_name = 'mistral-7b'; dataset = 'ultrachat'; M =  50_000
# model_name = 'llama-7b'; dataset = 'tulu_v1_mix'; M = 150_000


paths = glob.glob('curriculum/*/*/*/scores.pkl')
paths = [x for x in paths if 'llama' in x and 'tulu_v1_mix' in x and 'log_prob_neg' in x]
path = paths[0]

verbose = True
print(path)
pacing_fn = f'prune_size={M}_ep=3'
# pacing_fn = f'singlestep_size={M}_startingfrac=0.1'
# pacing_fn = f'singlestep_size={M}_startingfrac=0.2'
# pacing_fn = f'singlestep_size={M}_startingfrac=0.3'
# pacing_fn = f'fep_size={M}_nsteps=5_startingfrac=0.1_inc=2'
# pacing_fn = f'fep_size={M}_nsteps=5_startingfrac=0.1_inc=1.5'
# pacing_fn = f'fep_size={M}_nsteps=5_startingfrac=0.2_inc=1.5'
# pacing_fn = f'fep_size={M}_nsteps=5_startingfrac=0.1_inc=1.5'

pacing_fn_list = [
    f'prune_size={M}_ep=3',
    f'singlestep_size={M}_startingfrac=0.05',
#     f'singlestep_size={M}_startingfrac=0.1',
#     f'singlestep_size={M}_startingfrac=0.2',
#     f'singlestep_size={M}_startingfrac=0.3',
#     f'fep_size={M}_nsteps=5_startingfrac=0.1_inc=1.5',
#     f'fep_size={M}_nsteps=5_startingfrac=0.1_inc=2',
#     f'fep_size={M}_nsteps=5_startingfrac=0.1_inc=3',
    f'fep_size={M}_nsteps=5_startingfrac=0.05_inc=1.5',
    f'fep_size={M}_nsteps=5_startingfrac=0.05_inc=2',
#     f'fep_size={M}_nsteps=5_startingfrac=0.05_inc=3',
#     f'fep_size={M}_nsteps=5_startingfrac=0.05_inc=1.25'
]



nrows = len(pacing_fn_list)
fig, axs = plt.subplots(nrows, 3, figsize=(15,3*nrows), sharey=False, gridspec_kw={'width_ratios': [2,.5,.5]})

for i, pacing_fn in enumerate(pacing_fn_list):

    plt_kwargs = generate_curriculum(path, pacing_fn, verbose=True, save_output=False)
    output = plt_kwargs.pop('output')
    plt_kwargs.update({'fig': fig, 'axs': axs[i]})
    plt_curriculum(**plt_kwargs)

fig.tight_layout()



save_plt = 0
if save_plt:
    model_name, dataset, scoring_fn = output['model_name'], output['dataset'], output['scoring_fn']
    save_path = os.path.join(
        assets_dir, f'note_curriculum_{model_name}:{dataset}:{scoring_fn}.png')
    fig.savefig(save_path, bbox_inches='tight', dpi=100)



### main code

In [2]:
import os
import sys
import numpy as np
import time
import re
import random
import json
import pickle
from tqdm import tqdm 

import pyarrow
import torch
import transformers
import pandas as pd

from note_pruning import (
    save_to_pickle,
    save_sorted_inds,
    sort_kmeans_dist_to_cluster_centers,
    sort_dpp_map,
    save_prune_results,
    sort_dpp_map_memefficient,
)
from note_pruning_analysis import get_lm_output
from note_pruning_dpp import torch_vmf_kernel, torch_rbf_kernel
from functools import partial
from rosemary import parse_kv_from_string


In [3]:
test_run = False
dataset = 'tulu_v1_human_mix'
dataset = 'tulu_v2_human_mix'
dataset = 'flan_v2'
dataset = 'lima'
dataset = 'flan2022_1m'
dataset = 'tulu_v1_mix'
dataset = 'lima'
dataset = 'ultrachat200k'
dataset = 'wizardlm'
dataset = 'ultrachat15'

# sort_by = 'random_s=0'
# sort_by = 'kmeansl2_nc=3000'
# sort_by = 'kmeanscd_nc=3000'
# sort_by = 'log_prob'
# sort_by = 'dppmap_k=Kcos'
# sort_by = 'dppmap_k=Kcos1np'
# sort_by = 'el2n'
# sort_by = 'grad_norm'
# sort_by = 'kmeansl2_emb=grad+rp+loraB_nc=3000'
# sort_by = 'kmeansl2_emb=text+embedding_nc=3000'
sort_by = 'dppmap_emb=text+embedding_k=Kcos'
# sort_by = 'logit_margin'
# rhov1: mistral-7b base-tuned(ultrachat200k_beforesplitlongconv)
# sort_by = 'rhov1'
# sort_by = 'numtoks'
# sort_by = 'semdedup_cl=kmeansfaisscd_md=mpnet_dist=cd_emb=text+embedding_nc=200'

# used for generating model output.
# model_name = 'llama-7b'; encode_fn_type = 'sft'
# model_name = 'llama-7b_ft=hmv1'; encode_fn_type = 'sft'
# model_name = 'llama-7b+lora:r=256:a=256'; encode_fn_type = 'sft'
# model_name = 'mistral-7b+lora:r=256:a=256'; encode_fn_type = 'sft'
model_name = 'all-mpnet-base-v2'; encode_fn_type = 'input'
# model_name = 'bge-large-en-v1.5'; encode_fn_type = 'input'

# model_name = 'mistral-7b+lora:r=256:a=256__rho__mistral-7b-ultrachat200k-v1+lora:r=256:a=256'


save_dir = f"data_inds/"
save_dir = os.path.join(save_dir, '' if encode_fn_type=='sft' else encode_fn_type, model_name, dataset)
os.makedirs(save_dir, exist_ok=True) 

print(dataset, model_name, sort_by)
print(save_dir)

ultrachat15 all-mpnet-base-v2 dppmap_emb=text+embedding_k=Kcos
data_inds/input/all-mpnet-base-v2/ultrachat15


In [11]:

from glob import glob

paths = glob('results/*/*/eval/*/*.out')


def read_text(path):
    with open(path, 'r') as f:
        return f.read()
    
def read_text_bytes(path):
    with open(path, 'rb') as f:
        return f.read().decode('utf-8', 'ignore')


oom_eval_paths = []
for path in paths:
    try:
        s = read_text(path)
    except UnicodeDecodeError as e:
        print(e, '\n Re-try reading bytes')
        s = read_text_bytes(path)
    if 'CUDA out of memory' in s:
        oom_eval_paths.append(path)




'utf-8' codec can't decode byte 0xe2 in position 68208: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode byte 0xe2 in position 90650: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 67269-67270: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 83845-83846: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 68285-68286: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode byte 0xe2 in position 93620: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 81925-81926: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 107547-107548: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 80949-80950: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode byte 0xe2 in p

'utf-8' codec can't decode byte 0xe0 in position 118030: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode byte 0xe2 in position 81560: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode byte 0xe0 in position 36092: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 81261-81262: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode byte 0xe2 in position 77380: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 82587-82588: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 64119-64120: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 83205-83206: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 81915-81916: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode byte 0xe2 in posi

'utf-8' codec can't decode byte 0xe2 in position 100086: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 85105-85106: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 81243-81244: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode byte 0xe2 in position 107858: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode byte 0xe2 in position 84440: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 71173-71174: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode byte 0xe2 in position 83188: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 103595-103596: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode bytes in position 85079-85080: invalid continuation byte 
 Re-try reading bytes
'utf-8' codec can't decode byte 0xe2 in p

In [13]:
[x for x in oom_eval_paths if 'ft1_ep2' in x or 'oi' in x]

['results/oi4_tulu_v1_mix_ep=3/llama-7b_tuluv1m:50k_random_s=1/eval/alpacafarm_ann=chatgpt_chatfmt/1096790.out',
 'results/oi5_ultrachat15:mistral-7b/mistral-7b_ultrachat15_score=random:s=0_pace=prune:size=100000:ep=2/eval/gsm_s=8_cot_chatfmt/1170623.out',
 'results/oi5_ultrachat15:mistral-7b/mistral-7b_ultrachat15_score=random:s=0_pace=prune:size=100000:ep=2/eval/humaneval_chatfmt/1170626.out',
 'results/oi3/llama-7b_all:100k/eval/tydiqa_s=1_gp_chatfmt/798768.out',
 'results/oi3/llama-7b_all:100k/eval/tydiqa_s=1_gp/815636.out',
 'results/oi3/llama-7b_all:400k_humanmix/eval/tydiqa_s=1_gp_chatfmt/930364.out',
 'results/oi3/llama-7b_all:400k_humanmix/eval/tydiqa_s=1_gp/930355.out',
 'results/oi3/llama-7b_all:600k_humanmix/eval/tydiqa_s=1_gp_chatfmt/933798.out',
 'results/oi3/llama-7b_all:600k_humanmix/eval/tydiqa_s=1_gp/933789.out',
 'results/oi3/llama-7b_all:200k_humanmix/eval/tydiqa_s=1_gp_chatfmt/823171.out',
 'results/oi3/llama-7b_all:200k_humanmix/eval/tydiqa_s=1_gp/826095.out']

In [35]:
path = 'results/ft1_ep=2/llama-7b_cot/eval/humaneval/1108380.out'


with open(path, 'rb') as f:
    s = f.read()
    print(s.decode('utf-8', 'ignore'))
    

Running on dcs254
+ echo ======
+ srun python -m eval.codex_humaneval.run_eval --data_file data/eval/codex_humaneval/HumanEval.jsonl.gz --model_name_or_path results/ft1_ep=2/llama-7b_cot --save_dir results/ft1_ep=2/llama-7b_cot/eval/humaneval --eval_batch_size 10 --eval_pass_at_ks 1 --unbiased_sampling_size_n 1 --temperature 0.1
[2023-10-27 23:43:55,364] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Number of examples: 164
Loading model and tokenizer...
Loading checkpoint shards: 100%|██████████| 3/3 [00:22<00:00,  7.59s/it]
Sampling iter: 0 / 1
Generating Completions: 100%|██████████| 164/164 [16:29<00:00,  6.04s/it]
Reading samples...
164it [00:00, 20605.89it/s]
Running test suites...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly

In [18]:
path = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/oi5_ultrachat15:mistral-7b/mistral-7b_ultrachat15_score=random:s=0_pace=prune:size=100000:ep=2/eval/humaneval_chatfmt/1170626.out'
with open(path, 'r') as f:
    s = f.read()
    print('CUDA out of memory' in s)

True


In [6]:
p = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data_inds/all-mpnet-base-v2/ultrachat15/dppmapbd_nc=200_k=lin_kmd=mpnet_incr.pkl'

with open(p, 'rb') as f:
    d = pickle.load(f)

In [8]:
d['info']

{'dataset': 'ultrachat15',
 'kernel_type': 'lin',
 'kernel_embed_model': 'mpnet',
 'kernel_embed_type': 'text_embedding',
 'kernel_kwargs': {},
 'quality_score_type': None,
 'quality_score_embed_model': None,
 'theta': 0.0,
 'max_length': 5000,
 'M': 156326,
 'time_elapsed': 2720.3869178295135,
 'marginal_gains': [1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.0000001192092896,
  1.

In [6]:
d = get_lm_output(dataset, model_name, encode_fn_type=encode_fn_type, return_text_embedding=True)
if test_run:
    d = {k: v[:1000] for k, v in d.items()}
    
# some entries are nan, impute with mean value.
N = d['text_embedding'].shape[0]
print(N)

143000


In [336]:
## the copy in `note_pruning.py` is most up to date

pkl_extra = {}
inds = None

# sort_by = 'dppmap_theta=.5_k=vmf_gamma=1.0_kmd=mpnet_q=log+prob_qmd=llama7b'


# sort_by = 'dppmap_k=vmf_gamma=1.0_kmd=mpnet'
# sort_by = 'dppmap_k=vmf_gamma=0.3_kmd=mpnet'
# sort_by = 'dppmap_k=vmf_gamma=3.0_kmd=mpnet'
# sort_by = 'dppmap_theta=.5_k=vmf_gamma=3.0_kmd=mpnet_q=log+prob_qmd=mistral7b'

sort_by = 'dppmapbd_nc=200_k=vmf_gamma=0.3_kmd=mpnet'


t0 = time.time()
if any(sort_by.startswith(x) for x in [
        'log_prob', 
        'el2n',  # el2n_agg={l2n|mean}
        'logit_margin', 
        'grad',  # grad_{loraB|qkv|all|last}_l2n
    ]):
    if sort_by not in d:
        print(f'sort_by={sort_by} not in model output: ({dataset}, {model_name})')
    S = np.nan_to_num(d[sort_by], nan=np.nanmean(d[sort_by])).squeeze()
elif sort_by.startswith('random'):
    match = re.search(r's=(\d+)', sort_by)
    seed = int(match.group(1))
    np.random.seed(seed)
    S = np.random.rand(N)
    assert(S.shape == np.unique(S).shape)
if sort_by.startswith('kmeans'):
    dist_fn = 'l2' if sort_by.startswith('kmeansl2') else 'cd'
    match = re.search(r'nc=(\d+)', sort_by)
    n_clusters = int(match.group(1)) if match else None
    match = re.search(r'emb=([^_]+)', sort_by)
    embed_type = re.sub(r'[+]', '_', match.group(1)) if match else 'text_embedding'
    if embed_type not in set(d.keys()).intersection(set(['text_embedding', 'grad_rp_loraB'])):
        raise ValueError(f'Invalid embed_type = {embed_type}')
    emb = d[embed_type]
    print(f'Running kmeans(n_clusters={n_clusters}) {{ {embed_type} }} to compute {"euclidean" if dist_fn == "l2" else "cosine"} distance to cluster centers.')
    S, kms = sort_kmeans_dist_to_cluster_centers(emb, n_clusters, dist_fn=dist_fn)
    pkl_extra['kmeans'] = kms
elif sort_by.startswith('semdedup'):
    import note_pruning_clustering
    kvs = parse_kv_from_string(sort_by)
    md = kvs['md']
    if (md == 'mpnet' and model_name != 'all-mpnet-base-v2') or \
        (md == 'bge' and model_name != 'bge-large-en-v1.5') or \
        (md == 'llama7b' and not model_name.lower().startswith('llama-7b')) or \
        (md == 'mistral7b' and not model_name.lower().startswith('mistral-7b')):
        raise ValueError(f'md={md} does not match with model_name={model_name}')
    clustering_fn = create_string_from_kv(
        {k: v for k, v in kvs.items() if k in ['cl', 'nc', 'bsz', 'ms', 'emb']})
    dist = kvs['dist']
    assert(dist in ['cd', 'l2'])
    embed_type = re.sub(r'[+]', '_', kvs['emb'])
    save_dir_clustering = os.path.join('clustering', encode_fn_type, model_name, dataset, clustering_fn)
    os.makedirs(save_dir_clustering, exist_ok=True)
    # normalize embeddings to unit norm if the model that generated the embeddings does the 
    # same, e.g., mpnet, bge, or if using spherical kmeans clustering.
    if any(x in model_name for x in ['mpnet', 'bge']) or 'kmeansfaisscd' in clustering_fn:
        normalize_embeddings = True
    else:
        normalize_embeddings = False
    kwargs = {
        'model_name': model_name,
        'dataset': dataset,
        'encode_fn_type': encode_fn_type,
        'clustering_fn': clustering_fn,
        'embed_type': embed_type,
        'normalize_embeddings': normalize_embeddings,
        'first_N': None,
        'save_dir': save_dir_clustering,
    }
    print(f'Calling note_pruning_clustering.main with kwargs={json.dumps(kwargs, indent=4)}')
    X, Y, C = note_pruning_clustering.main(**kwargs)
    print('Apply SemDeDup to discard duplicates.')
    S = note_pruning_clustering.semdedup(X, Y, dist=dist, device='cuda')
elif sort_by.startswith('dpp_'):
    match = re.search(r'k=(\w+)', sort_by)
    kernel_type = match.group(1) if match else None
    match = re.search(r'emb=([^_]+)', sort_by)
    embed_type = re.sub(r'[+]', '_', match.group(1)) if match else 'text_embedding'
    if embed_type not in set(d.keys()).intersection(set(['text_embedding', 'grad_rp_loraB'])):
        raise ValueError(f'Invalid embed_type = {embed_type}')
    emb = d[embed_type]
    log_prob = d['log_prob']
    inds = sort_dpp_map_memefficient(emb, log_prob, kernel_type=kernel_type, torch_compile=False)
elif sort_by.startswith('dppmap_'):
    import note_pruning_dpp
    kvs = parse_kv_from_string(sort_by)
    if kvs['k'] == 'vmf':
        kernel_kwargs = {'gamma': kvs['gamma']}
    elif kvs['k'] == 'rbf':
        kernel_kwargs = {'sigma': kvs['sigma']}
    else:
        kernel_kwargs = {}
    kwargs = {
        'dppmap_type': 'dppmap',
        'dataset': dataset,
        'kernel_type': kvs['k'],
        'kernel_embed_model': kvs['kmd'],
        'kernel_embed_type': re.sub(r'[+]', '_', kvs['kemb']) if 'kemb' in kvs else 'text_embedding',
        'kernel_kwargs': kernel_kwargs,
        'quality_score_type': re.sub(r'[+]', '_', kvs['q']) if 'q' in kvs else None,
        'quality_score_embed_model': kvs.get('qmd', None),
        'theta': kvs.get('theta', 0.), # defaults to just diversity no quality
        'device': 'cuda',
        'max_length': 50, #min(50_000, .5*N), # balance finish job within 6 hrs with wanting to prune a lot
    }
    print(f'Calling note_pruning_dpp.compute_dppmap with kwargs={json.dumps(kwargs, indent=4)}')
    S, output = note_pruning_dpp.compute_dppmap(**kwargs)
    pkl_extra['info'] = output
elif sort_by.startswith('dppmapbd'):
    import note_pruning_clustering
    import note_pruning_dpp
    kvs = parse_kv_from_string(sort_by)
    md = kvs['kmd']
    if (md == 'mpnet' and model_name != 'all-mpnet-base-v2') or \
        (md == 'bge' and model_name != 'bge-large-en-v1.5') or \
        (md == 'llama7b' and not model_name.lower().startswith('llama-7b')) or \
        (md == 'mistral7b' and not model_name.lower().startswith('mistral-7b')):
        raise ValueError(f'md={md} does not match with model_name={model_name}')
    if kvs['k'] == 'vmf':
        kernel_kwargs = {'gamma': kvs['gamma']}
    elif kvs['k'] == 'rbf':
        kernel_kwargs = {'sigma': kvs['sigma']}
    else:
        kernel_kwargs = {}
    kwargs = {
        'dppmap_type': 'dppmapbd',
        'dataset': dataset,
        'kernel_type': kvs['k'],
        'kernel_embed_model': kvs['kmd'],
        'kernel_embed_type': re.sub(r'[+]', '_', kvs['kemb']) if 'kemb' in kvs else 'text_embedding',
        'kernel_kwargs': kernel_kwargs,
        'quality_score_type': re.sub(r'[+]', '_', kvs['q']) if 'q' in kvs else None,
        'quality_score_embed_model': kvs.get('qmd', None),
        'theta': kvs.get('theta', 0.), # defaults to just diversity no quality
        'device': 'cuda',
        'max_length': int(.6*N / kvs['nc']), 
    }
    clustering_fn = create_string_from_kv({
        'cl': kvs.get('cl', 'kmeansfaisscd'),
        'md': kvs['kmd'],
        'emb': kvs['kemb'] if 'kemb' in kvs else 'text+embedding',
        'nc': kvs['nc'],
    })
    save_dir_clustering = os.path.join(
        'clustering', encode_fn_type, model_name, dataset, clustering_fn)
    os.makedirs(save_dir_clustering, exist_ok=True)
    clustering_data_path = os.path.join(save_dir_clustering, 'data.pkl')
    if not os.path.isfile(clustering_data_path):
        normalize_embeddings = True if \
            (any(x in model_name for x in ['mpnet', 'bge']) or 'kmeansfaisscd' in clustering_fn) else False
        kwargs_clustering = {
            'model_name': model_name,
            'dataset': dataset,
            'encode_fn_type': encode_fn_type,
            'clustering_fn': clustering_fn,
            'embed_type': kwargs['kernel_embed_type'],
            'normalize_embeddings': normalize_embeddings,
            'first_N': None,
            'save_dir': save_dir_clustering,
        }
        print(f'Calling note_pruning_clustering.main with kwargs={json.dumps(kwargs_clustering, indent=4)}')
        X, Y, C = note_pruning_clustering.main(**kwargs_clustering)
    else:
        with open(clustering_data_path, 'rb') as f:
            data = pickle.load(f)
        Y = data['Y']
    print(f'Calling note_pruning_dpp.compute_dppmap with kwargs={json.dumps(kwargs, indent=4)}')
    kwargs.update({'Y': Y})
    S, output = note_pruning_dpp.compute_dppmap(**kwargs)
    pkl_extra['info'] = output
elif sort_by.startswith('rho'):
    if sort_by == 'rhov1':
        model_names = ['mistral-7b+lora:r=256:a=256',
                       'mistral-7b-ultrachat200k-v1+lora:r=256:a=256']
        assert(model_name == model_names[0])
    else:
        raise ValueError(f'sort_by={sort_by} not implemented.')
    assert(len(model_names) == 2)
    ds = []
    for x in model_names:
        ds.append(get_lm_output(dataset, x, return_text_embedding=False, fill_nan=False))
    ks = [set(d.keys()) for d in ds]
    ks = ks[0] & ks[1]
    for k in ks:
        S0 = ds[0][k]
        S1 = ds[1][k]
        # handle nan entries properly.
        nan_mask = np.logical_or(np.isnan(S0), np.isnan(S1))
        S = np.subtract(S0, S1)
        S[nan_mask] = np.nan
        S = S.squeeze()
        save_prune_results(save_dir, None, S, {}, f'{sort_by}_{k}', model_name, dataset)
elif sort_by.startswith('numtoks'):
    from transformers import AutoTokenizer
    from note_pruning_analysis import get_dataset_token_lengths
    if 'llama' in model_name or 'mistral' in model_name:
        tokenizer = AutoTokenizer.from_pretrained(
            '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/huggyllama/llama-7b',
            use_fast=False, # use_fast sometimes cause error.
        )
    else:
        raise ValueError('Need to supply appropriate tokenizer to count token lengths,')
    d = get_dataset_token_lengths(dataset, tokenizer)

    d['total_len'] = d['input_len'] + d['output_len']
    for k in ['input', 'output', 'total']:
        S = d[f'{k}_len']
        save_prune_results(save_dir, None, S, {}, f'{sort_by}_{k}', model_name, dataset)

        
t1 = time.time()
print(f'Rank datapoints with {sort_by} took {t1-t0:.2f} seconds.')


Calling note_pruning_dpp.compute_dppmap with kwargs={
    "dppmap_type": "dppmapbd",
    "dataset": "wizardlm",
    "kernel_type": "vmf",
    "kernel_embed_model": "mpnet",
    "kernel_embed_type": "text_embedding",
    "kernel_kwargs": {
        "gamma": 0.3
    },
    "quality_score_type": null,
    "quality_score_embed_model": null,
    "theta": 0.0,
    "device": "cuda",
    "max_length": 2250
}
dppmapbd: cluster = 0 / 200


 99%|█████████▉| 1526/1538 [00:52<00:00, 28.90it/s] 


Stop on dᵢ^2 = -3.0898661407263717e-08. len(inds)=1527 / 1539
Rank datapoints with dppmapbd_nc=200_k=vmf_gamma=0.3_kmd=mpnet took 53.88 seconds.


In [110]:

# from note_pruning import save_prune_results
if not any(sort_by.startswith(x) for x in ['rho', 'numtoks']):
    save_prune_results(save_dir, inds, S, pkl_extra, sort_by, model_name, dataset)

50000